## Combine 3 CSV files

In [28]:
import pandas as pd
import numpy as np

In [29]:
g = pd.read_csv("gdp_p_cap.csv").drop(["Unnamed: 0","index","Indicator Name","Indicator Code", "Unnamed: 64"],axis=1)
l = pd.read_csv("life_expectancy.csv").drop(["Unnamed: 0","index","Indicator Name","Indicator Code", "Unnamed: 64"],axis=1)
p = pd.read_csv("population.csv").drop(["Unnamed: 0","index","Indicator Name","Indicator Code", "Unnamed: 64"],axis=1)

In [30]:
g.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,412.014287,488.300251,543.303042,528.736648,576.190125,587.565090,583.656193,574.184114,571.073775,571.542506
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3775.592212,3928.458970,4094.358816,4209.886952,4276.620248,4327.392484,4413.115113,4524.684565,4683.743890,4867.916405
2,Algeria,DZA,2480.98563,2090.783287,1638.243986,2146.271842,2214.192522,2289.942015,2120.483825,2255.824179,...,4406.947221,4402.409201,4480.785777,4524.348982,4586.651663,4620.132179,4698.664711,4776.787543,4834.230203,4812.158456
3,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3652.153498,3549.577857,3587.883798,3579.960081,3748.449445,3796.882622,3843.198241,3748.320623,3530.309423,3409.929285
4,Antigua and Barbuda,ATG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16545.776949,14316.338568,13092.073818,12643.919844,12919.954160,12750.414275,13193.546602,13577.412803,14190.605760,14483.208237


In [31]:
years = [str(x) for x in range(1960,2018)]
g = g[years+ ["Country Name", "Country Code"]]
l = l[years+ ["Country Name", "Country Code"]]
p = p[years+ ["Country Name", "Country Code"]]

In [32]:
filled_g = g.T.mask(g.T.ffill().isnull(), 0.0).T
filled_l = l.T.mask(l.T.ffill().isnull(), 0.0).T
filled_p = p.T.mask(p.T.ffill().isnull(), 0.0).T

In [33]:
set_g = set(filled_g.loc[filled_g.isnull().sum(1)>5].index)
set_l = set(filled_l.loc[filled_l.isnull().sum(1)>5].index)
set_p = set(filled_p.loc[filled_p.isnull().sum(1)>5].index)

In [34]:
rows_to_remove = set_g.union(set_l).union(set_p)

In [35]:
new_g = filled_g.drop(rows_to_remove).drop("Country Code",axis=1).reset_index()
new_l = filled_l.drop(rows_to_remove).drop("Country Code",axis=1).reset_index()
new_p = filled_p.drop(rows_to_remove).reset_index()

In [36]:
merged_tmp = pd.merge(left=new_g,right=new_l, \
                        left_on='Country Name', right_on='Country Name',\
                        suffixes=('_gdp', '_life'),how="inner")
data = pd.merge(left=merged_tmp, right=new_p, \
                        left_on='Country Name', right_on='Country Name',\
                        suffixes=('', '_pop'),how="inner")

## Combine Country Code with Continent

In [44]:
continent = pd.read_csv("country-and-continent-codes-list.csv")

In [45]:
data_final = pd.merge(left=data, right=continent, \
                        left_on='Country Name', right_on='Three_Letter_Country_Code',\
                        how="inner")

In [49]:
data_final.to_csv("data.csv")